In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/sriwijaya-air/?sortby=post_date%3ADesc&pagesize=50'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    try:
        review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    
    except:
        review_dict['Overall_rating'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'sriwijaya air'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/sriwijaya_air.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
45,sriwijaya air,Fitriansyah Yudha,2014-07-29,PLM-CGK with Boeing 737-500. The flight was on...,NaN,Economy Class,7,3,3,3,1,NaN,5,yes
46,sriwijaya air,H Bama,2014-04-06,Went from DPS to DIL with old Boeing 737. My t...,NaN,Economy Class,7,NaN,NaN,NaN,NaN,NaN,3,yes
47,sriwijaya air,D Olsson,2014-03-10,I often fly Dili-Denpasar. In comparison to Me...,NaN,Economy Class,5,NaN,NaN,NaN,NaN,NaN,3,no
48,sriwijaya air,H Phillips,2013-08-22,Flew CGK-SIN with Sriwijaya Air to make a chan...,NaN,Economy Class,7,4,5,5,3,NaN,5,yes
49,sriwijaya air,Bayu Muhardika,2013-06-19,Flight SJ099 from TKG-CGK was planned at 16.40...,NaN,Economy Class,NaN,3,2,2,1,NaN,1,no


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,sriwijaya air,R Borsok,2020-03-22,✅ Trip Verified | Absolute disaster. First af...,Couple Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
1,sriwijaya air,C Mernele,2020-02-01,✅ Trip Verified | Sorong to Ambon. I bought a...,Couple Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
2,sriwijaya air,Karsten Busche,2019-11-14,Not Verified | Siborong-Borong to Jakarta. Ho...,Couple Leisure,Economy Class,1,1,1,1,1,1,1,no
3,sriwijaya air,Eduard Renom,2019-10-10,✅ Trip Verified | We can’t believe that they h...,Couple Leisure,Economy Class,1,1,1,1,1,1,1,no
4,sriwijaya air,S Garlen,2019-08-23,✅ Trip Verified | \r\n \r\nSorong to Jayapur...,Solo Leisure,Economy Class,7,5,4,2,NaN,4,4,yes
